In [1]:
import numpy as np
from kifmm_py import KiFmm

# For testing consistency
np.random.seed(0)

# Laplace

In [2]:
dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
expansion_order = 6
n_vec = 1
n_crit = 150
sparse = True
n_sources = 1000
n_targets = 2000
kernel = "laplace"  # Or 'helmholtz'
field_translation = "blas"  # Or 'fft'
kernel_eval_type = (
    "eval_deriv"  # For potential graidents, or 'eval' for potentials only
)

# Setup source/target/charge data in Fortran order
sources = np.zeros((n_sources, dim), order="F").astype(dtype)
sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim), order="F"
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim), order="F"
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec), order="F"
).astype(dtype)

In [3]:
fmm = KiFmm(
    expansion_order,
    sources,
    targets,
    charges,
    n_crit,
    sparse,
    kernel_eval_type,
    kernel,
    field_translation,
    svd_threshold=1e-7,  # Optional, required for 'blas' based field translations
)

In [4]:
# Run the FMM
fmm.evaluate()

In [5]:
# Examine result at leaf index
leaf = fmm.target_leaves[1]
found = fmm.potentials(leaf)[0]
targets_leaf = fmm.target_coordinates(leaf)
expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

# Test
np.testing.assert_allclose(found, expected, rtol=1e-2)

# Helmholtz

In [6]:
# Set FMM Parameters
kernel = "helmholtz"  # Or 'helmholtz'
field_translation = "fft"  # Or 'blas'
kernel_eval_type = "eval"  # For potentials only

# Now need complex charges
charges = np.array(np.ones((n_sources, 1)).astype(ctype), order="F")
for i, c in enumerate(charges):
    charges[i] = i * (1 + 1j)

In [7]:
fmm = KiFmm(
    expansion_order,
    sources,
    targets,
    charges,
    n_crit,
    sparse,
    kernel_eval_type,
    kernel,
    field_translation,
    wavenumber=1.0,  # Optional, required for 'helmholtz' FMM
)

In [8]:
# Run the FMM
fmm.evaluate()

# Examine result at leaf index
leaf = fmm.target_leaves[1]
found = fmm.potentials(leaf)[0]
targets_leaf = fmm.target_coordinates(leaf)
expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

# Test
np.testing.assert_allclose(found, expected, rtol=1e-5)

# Multipole Charge Vectors With BLAS field translations

In [9]:
n_vec = 5
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec), order="F"
).astype(dtype)

# Set FMM Parameters
expansion_order = 6
n_vec = 1
n_crit = 150
sparse = True
n_sources = 1000
n_targets = 2000
kernel = "laplace"  # Or 'helmholtz'
field_translation = (
    "blas"  # only blas field translations permitted for multiple charge vectors
)
kernel_eval_type = "eval"


fmm = KiFmm(
    expansion_order,
    sources,
    targets,
    charges,
    n_crit,
    sparse,
    kernel_eval_type,
    kernel,
    field_translation,
    svd_threshold=1e-7,  # Optional, required for 'blas' based field translations
)

# As before evaluate
fmm.evaluate()